### Decorators Application (Logger, Stacked Decorators)

В этом примере мы создадим декоратор утилиты, который будет регистрировать вызовы функций (в консоли, но на практике вы бы записывали свои журналы в файл (например, используя встроенный регистратор Python) или в базу данных и т. д.

In [1]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result

    return inner

In [2]:
@logged
def func_1():
    pass

In [3]:
@logged
def func_2():
    pass

In [4]:
func_1()

func_1: called 2017-12-10 00:09:19.443657+00:00


In [5]:
func_2()

func_2: called 2017-12-10 00:09:19.460691+00:00


Теперь мы можем дополнительно также захотеть засекать время функции. Мы, конечно, можем включить код для этого в наш `logged` декоратор, но мы также могли бы просто использовать декоратор `@timed`, который мы уже написали, **складывая** наши декораторы.

In [6]:
def timed(fn):
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result

    return inner

In [7]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [8]:
factorial(10)

factorial: called 2017-12-10 00:09:19.496762+00:00
factorial ran for 0.000130s


3628800

Обратите внимание, что порядок, в котором мы размещаем декораторы, может иметь значение!

Помните, что это произошло потому, что наши многоярусные декораторы по сути представляли собой:

In [9]:
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

Поэтому в этом случае сначала будет вызван декоратор `timed`, а затем декоратор `logged`.

Вы можете удивиться, почему выводимые данные выглядят перевернутыми. Посмотрите, как были определены декораторы — сначала они запускали переданную функцию, а затем **затем** выводили результат.

Итак, в приведенном выше примере упрощенный взгляд на то, что происходит в каждом декораторе:

* `timed(fn)(*args, **kwargs)`:

1. вызывает `fn(*args, **kwargs)`
2. выводит время

* `logged(fn)(*args, **kwargs)`:

1. вызывает `fn(*args, **kwargs)`
2. выводит информацию журнала

Итак, вызов
`factorial = timed(logged(factorial))`

эквивалентно:

<pre>
fn = logged(factorial)
factorial = timed(fn)

factorial(n) --> вызывает timed(fn)(n)
--> вызывает fn(n), затем выводит время
--> вызывает logged(original_factorial)(n), затем выводит время
--> вызывает original_factorial(n), затем регистрирует, затем выводит синхронизация
</pre>

Итак, как вы видите, сначала запустился декоратор `timed`, но сначала он вызвал декорированную функцию logged, а затем вывел результат — поэтому вывод на печать выглядит перевернутым.

In [10]:
factorial(10)

factorial: called 2017-12-10 00:09:19.525820+00:00
factorial ran for 0.000147s


3628800

Но в следующем случае сначала будет запущен декоратор `logged`, а затем декоратор `timed`:

In [11]:
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

In [12]:
factorial(10)

factorial ran for 0.000015s
factorial: called 2017-12-10 00:09:19.547866+00:00


3628800

Или, используя обозначение **@**:

In [13]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [14]:
factorial(10)

factorial ran for 0.000016s
factorial: called 2017-12-10 00:09:19.572914+00:00


3628800

In [15]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [16]:
factorial(10)

factorial: called 2017-12-10 00:09:19.608237+00:00
factorial ran for 0.000153s


3628800

Чтобы прояснить ситуацию, давайте напишем два очень простых декоратора следующим образом:

In [17]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

In [18]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [19]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [20]:
my_func()

running dec_1
running dec_2
running my_func


Но если мы изменим порядок декораторов:

In [21]:
@dec_2
@dec_1
def my_func():
    print('running my_func')

In [22]:
my_func()

running dec_2
running dec_1
running my_func


Вы можете задаться вопросом, имеет ли это значение на практике. И да, имеет.

Рассмотрим API, содержащий различные функции, которые могут быть вызваны. Однако конечные точки защищены и могут быть запущены только аутентифицированными пользователями, имеющими некоторые определенные роли. Если у них нет роли, вы хотите вернуть несанкционированную ошибку. Но если она есть, то вы хотите зарегистрировать, что они вызвали конечную точку.

В этом случае у вас может быть один декоратор, который используется для проверки аутентификации и разрешений (и немедленно возвращает несанкционированную ошибку из API, если применимо), а другой — для регистрации вызова.

Если вы декорируете его следующим образом:

<pre>
@log
@authorize
def my_endpoint():
    pass
</pre>

тогда вызов всегда будет зарегистрирован.

Но в этом случае:

<pre>
@authorize
@log
def my_endpoint():
    pass
</pre>

ваша конечная точка будет зарегистрирована только в том случае, если пользователь прошел тест `authorize`.